In [36]:
import numpy as np
import pickle
import tensorflow as tf

In [42]:
import numpy as np
import pickle

def split_user_embeddings(user_emb, meta):
    """
    Split the 71-dim user embeddings into X_A (39) and X_T (32)
    
    Returns:
        X_A: (n_users, 39) - Explicit user attributes
        X_T: (n_users, 32) - Derived user attributes
    """
    user_ids = meta['user_ids']
    n_users = len(user_ids)
    
    # Extract embeddings in correct user order
    user_embedding_matrix = np.zeros((n_users, 71))
    for idx, user_id in enumerate(user_ids):
        if user_id in user_emb['user_embeddings']:
            user_embedding_matrix[idx] = user_emb['user_embeddings'][user_id]
        else:
            print(f"⚠️  Warning: User {user_id} not found in embeddings")
    
    # Split into X_A and X_T
    X_A = user_embedding_matrix[:, :39]   # First 39 dims = explicit
    X_T = user_embedding_matrix[:, 39:]   # Last 32 dims = derived
    
    print(f"\n📊 User embeddings split:")
    print(f"  X_A (explicit): {X_A.shape}")
    print(f"  X_T (derived):  {X_T.shape}")
    print(f"  Total:          {user_embedding_matrix.shape}")
    
    return X_A, X_T


def construct_P_l_correct(user_emb, user_pers, meta, level):
    """
    Construct P^l = [X_A || X_T || A^l_u]
    
    This matches the structure of Q^l = [Y_A || Y_T || A^l_p]
    """
    level_key = f'level_{level}'
    
    # Split user embeddings
    X_A, X_T = split_user_embeddings(user_emb, meta)
    
    # Get inter-level user features
    A_lu = user_pers['A_lu'][level_key]
    
    # Concatenate: P^l = [X_A || X_T || A^l_u]
    P_l = np.concatenate([X_A, X_T, A_lu], axis=1)
    
    print(f"\n🔨 Level {level} - P^l construction:")
    print(f"  X_A:      {X_A.shape}")
    print(f"  X_T:      {X_T.shape}")
    print(f"  A^l_u:    {A_lu.shape}")
    print(f"  ─────────────────────────")
    print(f"  P^l:      {P_l.shape}")
    
    return P_l, (X_A, X_T, A_lu)


def verify_Q_l_structure(poi_emb, poi_inter, level):
    """
    Verify Q^l = [Y_A || Y_T || A^l_p] structure
    """
    level_key = f'level_{level}'
    
    Y_A = poi_emb['poi_embeddings'][level_key]['Y_A']
    Y_T = poi_emb['poi_embeddings'][level_key]['Y_T']
    A_lp = poi_inter['A_lp'][level_key]
    Q_l = poi_inter['Q_l'][level_key]
    
    # Verify concatenation
    Q_l_reconstructed = np.concatenate([Y_A, Y_T, A_lp], axis=1)
    
    if np.allclose(Q_l, Q_l_reconstructed):
        print(f"✓ Q^l verified")
    else:
        print(f"⚠️  Q^l structure mismatch")
    
    print(f"\n🔨 Level {level} - Q^l structure:")
    print(f"  Y_A:      {Y_A.shape}")
    print(f"  Y_T:      {Y_T.shape}")
    print(f"  A^l_p:    {A_lp.shape}")
    print(f"  ─────────────────────────")
    print(f"  Q^l:      {Q_l.shape}")
    
    return Q_l, (Y_A, Y_T, A_lp)


def build_feature_checkin_matrices_correct(user_emb, user_pers, poi_emb, poi_inter, meta):
    """
    Build S^l = P^l @ (Q^l)^T with correct component concatenation
    
    P^l = [X_A || X_T || A^l_u]  (User: explicit || derived || inter-level)
    Q^l = [Y_A || Y_T || A^l_p]  (POI:  explicit || derived || inter-level)
    
    S^l = P^l @ (Q^l)^T          (Feature-based check-in matrix)
    """
    S_matrices = {}
    P_components = {}
    Q_components = {}
    dimension_info = {}
    
    for level in range(4):
        print(f"\n{'='*70}")
        print(f"{'LEVEL ' + str(level):^70}")
        print(f"{'='*70}")
        
        level_key = f'level_{level}'
        
        # Construct P^l
        P_l, (X_A, X_T, A_lu) = construct_P_l_correct(user_emb, user_pers, meta, level)
        
        # Verify Q^l
        Q_l, (Y_A, Y_T, A_lp) = verify_Q_l_structure(poi_emb, poi_inter, level)
        
        # Check dimension compatibility
        print(f"\n📐 Dimension check:")
        print(f"  P^{level}: {P_l.shape[1]} features")
        print(f"  Q^{level}: {Q_l.shape[1]} features")
        
        if P_l.shape[1] == Q_l.shape[1]:
            print(f"  ✓ Dimensions match!")
        else:
            print(f"  ✗ Dimension mismatch!")
            print(f"\n  P^{level} components: {X_A.shape[1]} + {X_T.shape[1]} + {A_lu.shape[1]} = {P_l.shape[1]}")
            print(f"  Q^{level} components: {Y_A.shape[1]} + {Y_T.shape[1]} + {A_lp.shape[1]} = {Q_l.shape[1]}")
            
            # Adjust if needed
            if P_l.shape[1] < Q_l.shape[1]:
                padding = np.zeros((P_l.shape[0], Q_l.shape[1] - P_l.shape[1]))
                P_l = np.concatenate([P_l, padding], axis=1)
                print(f"  → Padded P^{level} to {P_l.shape}")
            elif P_l.shape[1] > Q_l.shape[1]:
                P_l = P_l[:, :Q_l.shape[1]]
                print(f"  → Truncated P^{level} to {P_l.shape}")
        
        # Compute S^l = P^l @ (Q^l)^T
        print(f"\n🔢 Computing S^{level} = P^{level} @ (Q^{level})^T")
        S_l = np.matmul(P_l, Q_l.T)
        
        # Validate
        assert not np.isnan(S_l).any(), f"Level {level}: NaN values detected!"
        assert not np.isinf(S_l).any(), f"Level {level}: Inf values detected!"
        
        # Statistics
        print(f"\n📊 S^{level} statistics:")
        print(f"  Shape:        {S_l.shape[0]:>5} users × {S_l.shape[1]:>5} POIs")
        print(f"  Range:        [{S_l.min():>8.4f}, {S_l.max():>8.4f}]")
        print(f"  Mean ± Std:   {S_l.mean():>8.4f} ± {S_l.std():.4f}")
        print(f"  Median:       {np.median(S_l):>8.4f}")
        print(f"  Non-zero:     {np.count_nonzero(S_l):>8} / {S_l.size}")
        
        # Store results
        S_matrices[level_key] = S_l
        P_components[level_key] = {'X_A': X_A, 'X_T': X_T, 'A_lu': A_lu, 'P_l': P_l}
        Q_components[level_key] = {'Y_A': Y_A, 'Y_T': Y_T, 'A_lp': A_lp, 'Q_l': Q_l}
        
        dimension_info[level_key] = {
            'n_users': P_l.shape[0],
            'n_pois': Q_l.shape[1],
            'P_l_dims': {
                'X_A': X_A.shape[1],
                'X_T': X_T.shape[1],
                'A_lu': A_lu.shape[1],
                'total': P_l.shape[1]
            },
            'Q_l_dims': {
                'Y_A': Y_A.shape[1],
                'Y_T': Y_T.shape[1],
                'A_lp': A_lp.shape[1],
                'total': Q_l.shape[1]
            },
            'S_l_shape': S_l.shape,
            'S_l_stats': {
                'min': float(S_l.min()),
                'max': float(S_l.max()),
                'mean': float(S_l.mean()),
                'std': float(S_l.std()),
                'median': float(np.median(S_l))
            }
        }
    
    return S_matrices, P_components, Q_components, dimension_info


# ============================================================
# EXECUTE
# ============================================================

print("="*70)
print(" "*15 + "FEATURE-BASED CHECK-IN MATRIX COMPUTATION")
print("="*70)

S_matrices, P_components, Q_components, dimension_info = build_feature_checkin_matrices_correct(
    user_emb, user_pers, poi_emb, poi_inter, meta
)

# ============================================================
# SAVE RESULTS
# ============================================================

output = {
    'S_matrices': S_matrices,
    'P_components': P_components,
    'Q_components': Q_components,
    'dimension_info': dimension_info,
    'metadata': {
        'formula': 'S^l = P^l @ (Q^l)^T',
        'P_l_formula': 'P^l = [X_A || X_T || A^l_u]',
        'Q_l_formula': 'Q^l = [Y_A || Y_T || A^l_p]',
        'X_A_dims': 39,
        'X_T_dims': 32,
        'user_embedding_dims': 71,
        'created_at': str(np.datetime64('now')),
        'levels': [0, 1, 2, 3]
    }
}

with open('feature_checkin_matrices.pkl', 'wb') as f:
    pickle.dump(output, f)

print("\n" + "="*70)
print("✅ SUCCESS: Feature-based check-in matrices computed and saved!")
print("="*70)

print("\n📁 Saved to: feature_checkin_matrices.pkl")
print("\n📋 Summary:")
for level in range(4):
    info = dimension_info[f'level_{level}']
    print(f"  Level {level}: {info['n_users']:>2} users × {info['n_pois']:>4} POIs, "
          f"scores [{info['S_l_stats']['min']:>7.2f}, {info['S_l_stats']['max']:>7.2f}]")

print("\n" + "="*70)

               FEATURE-BASED CHECK-IN MATRIX COMPUTATION

                               LEVEL 0                                

📊 User embeddings split:
  X_A (explicit): (21, 39)
  X_T (derived):  (21, 32)
  Total:          (21, 71)

🔨 Level 0 - P^l construction:
  X_A:      (21, 39)
  X_T:      (21, 32)
  A^l_u:    (21, 221)
  ─────────────────────────
  P^l:      (21, 292)
✓ Q^l verified

🔨 Level 0 - Q^l structure:
  Y_A:      (4696, 149)
  Y_T:      (4696, 72)
  A^l_p:    (4696, 221)
  ─────────────────────────
  Q^l:      (4696, 442)

📐 Dimension check:
  P^0: 292 features
  Q^0: 442 features
  ✗ Dimension mismatch!

  P^0 components: 39 + 32 + 221 = 292
  Q^0 components: 149 + 72 + 221 = 442
  → Padded P^0 to (21, 442)

🔢 Computing S^0 = P^0 @ (Q^0)^T

📊 S^0 statistics:
  Shape:           21 users ×  4696 POIs
  Range:        [ -4.2910,  11.9947]
  Mean ± Std:     0.7048 ± 1.3809
  Median:         0.5723
  Non-zero:        98616 / 98616

                               LEVEL 1  